Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
from torch.cuda.amp import GradScaler, autocast

print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

/home/lundm/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True
cuda


Data Transformations and Dataset file paths

In [2]:
img_train_path = 'images/train'
img_validation_path = 'images/validation'

In [3]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(224),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load datasets
train_dataset = datasets.ImageFolder(root=img_train_path, transform=transform)
val_dataset = datasets.ImageFolder(root=img_validation_path, transform=transform)

batch_size = 128

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

Model Loading (Utilizing ResNet18)

In [4]:
class EmotionCNN(nn.Module):
    def __init__(self, num_classes):
        super(EmotionCNN, self).__init__()
        self.model = models.resnet50(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
    
    def forward(self, x):
        return self.model(x)

Training + Loss Func + Optimizer

In [5]:
num_classes = 7

model = EmotionCNN(num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(model.parameters(), lr=2e-4)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)
scaler = GradScaler()


num_epochs = 15

/home/lundm/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/lundm/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    corrects = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        corrects += torch.sum(preds == labels.data)
    
    scheduler.step()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = corrects.double() / len(train_loader.dataset)
    print(f'Epoch {epoch}/{num_epochs - 1}, Training Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_acc:.4f}')

    # Validation loop
    model.eval()
    val_loss = 0.0
    corrects = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            corrects += torch.sum(preds == labels.data)
    
    val_loss = val_loss / len(val_loader.dataset)
    val_acc = corrects.double() / len(val_loader.dataset)
    print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}')

Epoch 0/14, Training Loss: 1.4989, Training Accuracy: 0.4133
Validation Loss: 1.3814, Validation Accuracy: 0.4711
Epoch 1/14, Training Loss: 1.3398, Training Accuracy: 0.4821
Validation Loss: 1.2986, Validation Accuracy: 0.5001
Epoch 2/14, Training Loss: 1.2877, Training Accuracy: 0.5032
Validation Loss: 1.2637, Validation Accuracy: 0.5146
Epoch 3/14, Training Loss: 1.2579, Training Accuracy: 0.5163
Validation Loss: 1.2479, Validation Accuracy: 0.5212
Epoch 4/14, Training Loss: 1.2243, Training Accuracy: 0.5325
Validation Loss: 1.2367, Validation Accuracy: 0.5304
Epoch 5/14, Training Loss: 1.2069, Training Accuracy: 0.5419
Validation Loss: 1.2046, Validation Accuracy: 0.5413
Epoch 6/14, Training Loss: 1.1915, Training Accuracy: 0.5455
Validation Loss: 1.1948, Validation Accuracy: 0.5413
Epoch 7/14, Training Loss: 1.1873, Training Accuracy: 0.5460
Validation Loss: 1.1970, Validation Accuracy: 0.5514
Epoch 8/14, Training Loss: 1.1771, Training Accuracy: 0.5516
Validation Loss: 1.1949, Va